In [709]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer


In [710]:
# Read the data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')


In [711]:
#Determine if there is any columns with NaN values

missing_cols = df_train.columns[df_train.isna().any()].tolist()
print(missing_cols)

# Almost all the columns have NaN values, so we need to determine the percentage of NaN values in each column

def count_nans(df):
    for col in df.columns:
        print(col, df[col].isna().sum()/len(df[col])*100)

count_nans(df_train)

df_train = df_train[df_train['Cabin'].notna()]
df_train["Transported"] = df_train["Transported"].astype(int)

df_train = df_train.drop(["Name"], axis=1)

X_test = df_test.copy()
X_test = X_test.drop(["PassengerId"], axis=1)
X_test = X_test.drop(["Name"], axis=1)

['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name']
PassengerId 0.0
HomePlanet 2.312205222592891
CryoSleep 2.4962613597147127
Cabin 2.289198205452663
Destination 2.093638559760727
Age 2.0591280340503855
VIP 2.3352122397331185
RoomService 2.082135051190613
FoodCourt 2.105142068330841
ShoppingMall 2.392729782583688
Spa 2.105142068330841
VRDeck 2.1626596111814105
Name 2.300701714022777
Transported 0.0


In [712]:
# Select the target
y = df_train.Transported
# Select the features
X = df_train.drop(['Transported', "PassengerId"], axis=1)

#Divide the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [713]:
# Define the MAE function to compare the performance

def mae(X_train, X_valid, y_train, y_valid, model):
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions)
    
    return accuracy

In [714]:
# print(X_train.sample(10))
# print(X_valid.sample(3))

def count_nans(df):
    for col in df.columns:
        print(col, df[col].isna().sum()/len(df[col])*100)

count_nans(X_train)

HomePlanet 2.2222222222222223
CryoSleep 2.6342899190581313
Cabin 0.0
Destination 2.0897718910963947
Age 2.0456217807211186
VIP 2.251655629139073
RoomService 2.0309050772626933
FoodCourt 2.0161883738042676
ShoppingMall 2.369389256806475
Spa 2.0161883738042676
VRDeck 2.1486387049300957


In [715]:
# Configure the column Cabin. \d - any digit
def cabin_conf(input):
    input = str(input)
    cabin_conf_pattern = r"/\d+/"
    output = re.sub(cabin_conf_pattern, "/", input)
    return output

# Now I want to divide it into two different columns 
X_train["Cabin"] = X_train["Cabin"].apply(cabin_conf)
X_valid["Cabin"] = X_valid["Cabin"].apply(cabin_conf)
X_test["Cabin"] = X_test["Cabin"].apply(cabin_conf)


X_train[['Cabin_Deck', 'Cabin_Side']] = X_train['Cabin'].str.split('/', expand=True)
X_valid[['Cabin_Deck', 'Cabin_Side']] = X_valid['Cabin'].str.split('/', expand=True)
X_test[['Cabin_Deck', 'Cabin_Side']] = X_test['Cabin'].str.split('/', expand=True)

X_train.drop(["Cabin"], axis=1)
X_valid.drop(["Cabin"], axis=1)
X_test.drop(["Cabin"], axis=1)


print(X_train.sample(10))


     HomePlanet CryoSleep Cabin    Destination   Age    VIP  RoomService  \
6111     Europa      True   C/P    55 Cancri e  42.0  False          0.0   
7232     Europa      True   C/P    55 Cancri e  20.0  False          0.0   
3961       Mars     False   F/S    55 Cancri e  19.0  False       1184.0   
180      Europa      True   B/P    55 Cancri e  45.0  False          0.0   
4567      Earth     False   F/S    TRAPPIST-1e  17.0  False          0.0   
2945      Earth     False   G/S  PSO J318.5-22  60.0  False          0.0   
4527     Europa     False   C/S    55 Cancri e  49.0  False          0.0   
1296      Earth     False   F/P    TRAPPIST-1e  22.0  False        926.0   
3456      Earth      True   G/P    TRAPPIST-1e  16.0  False          0.0   
1924      Earth      True   G/P  PSO J318.5-22  18.0  False          0.0   

      FoodCourt  ShoppingMall    Spa  VRDeck Cabin_Deck Cabin_Side  
6111        0.0           0.0    0.0     0.0          C          P  
7232        0.0          

In [716]:

nunique_cnt = X_train.nunique()
print(nunique_cnt)
low_card_cols = nunique_cnt[nunique_cnt < 20].index.tolist()
print("The following columns will be translated into categorical: " + str(low_card_cols))

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train[low_card_cols] = ordinal_encoder.fit_transform(X_train[low_card_cols])
X_valid[low_card_cols] = ordinal_encoder.transform(X_valid[low_card_cols])

X_test[low_card_cols] = ordinal_encoder.fit_transform(X_test[low_card_cols])

HomePlanet         3
CryoSleep          2
Cabin             15
Destination        3
Age               80
VIP                2
RoomService     1096
FoodCourt       1283
ShoppingMall     968
Spa             1162
VRDeck          1127
Cabin_Deck         8
Cabin_Side         2
dtype: int64
The following columns will be translated into categorical: ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Cabin_Deck', 'Cabin_Side']


In [717]:
# Use imputer for ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Cabin_Deck', 'Cabin_Side']] columns

cols_impute = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Cabin_Deck', 'Cabin_Side']

my_imputer = SimpleImputer(strategy='most_frequent')
X_train_imputed = pd.DataFrame(my_imputer.fit_transform(X_train))
X_valid_imputed = pd.DataFrame(my_imputer.transform(X_valid))

X_test_imputed = pd.DataFrame(my_imputer.transform(X_test))

X_train_imputed.columns = X_train.columns
X_valid_imputed.columns = X_valid.columns
X_test_imputed.columns = X_test.columns


X_train = X_train_imputed
X_valid = X_valid_imputed
X_test = X_test_imputed

count_nans(X_train)

print(X_train.sample(10))

HomePlanet 0.0
CryoSleep 0.0
Cabin 0.0
Destination 0.0
Age 0.0
VIP 0.0
RoomService 0.0
FoodCourt 0.0
ShoppingMall 0.0
Spa 0.0
VRDeck 0.0
Cabin_Deck 0.0
Cabin_Side 0.0
      HomePlanet  CryoSleep  Cabin  Destination   Age  VIP  RoomService  \
1263         1.0        0.0    5.0          2.0  52.0  0.0          2.0   
158          0.0        0.0   11.0          2.0  51.0  0.0        172.0   
5503         2.0        0.0   10.0          2.0  38.0  0.0        490.0   
124          2.0        1.0   11.0          2.0  33.0  0.0          0.0   
4012         0.0        0.0   10.0          2.0  64.0  0.0          0.0   
6123         0.0        0.0   13.0          2.0  38.0  0.0          0.0   
5549         0.0        0.0   10.0          2.0  20.0  0.0        127.0   
6544         0.0        0.0   11.0          2.0  27.0  0.0          0.0   
5905         0.0        0.0   11.0          2.0  26.0  0.0        401.0   
4331         0.0        0.0   11.0          0.0  37.0  0.0          0.0   

      F

In [721]:
model_rfc = RandomForestClassifier(n_estimators=500)
print(mae(X_train, X_valid, y_train, y_valid, model_rfc))

model_rfc.fit(X_train, y_train)
predictions = model_rfc.predict(X_test)

predictions = predictions.astype(bool)

0.7969393761035903


In [722]:
# Create the model
# model = XGBRegressor(n_estimators=800,
#                          learning_rate=0.05) 
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Transported': predictions})
output.to_csv('submission.csv', index=False)

print(len(output.index))

4277
